In [1]:
import os
import sys
import re
import time
import matplotlib
import matplotlib.pyplot as plt
import random

parentPath = os.path.abspath("..")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)

import model as modellib
import visualize
from model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = parentPath

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# WEIGHTS_DIR = os.path.join(MODEL_DIR, "seg_ade20k20171109T1726")
WEIGHTS_DIR = os.path.join(MODEL_DIR, "seg_scenenn20171109T1726")

SCENENN_DIR = "/external_datasets/sceneNN"
# ADE20K_DIR = "/external_datasets/ADE20K_2016_07_26"

Using TensorFlow backend.


In [2]:
# import sceneNN
from instance_segmentation.sceneNN.dataset import *
# from instance_segmentation.ADE20K.dataset import *

In [3]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [4]:
# Validation dataset
dataset_test = ObjectsDataset()
dataset_test.load(SCENENN_DIR, 'testing', skip=9)
# dataset_test.load(ADE20K_DIR, 'validation')
dataset_test.prepare()

added 2621 images for testing


## Detection

In [5]:
class InferenceConfig(ObjectsConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=MODEL_DIR)

In [ ]:
mAPs = []
count = 0
for i, file in enumerate(sorted(os.listdir(WEIGHTS_DIR))):
    if file[-3:] != '.h5':
        continue
    if count % 2 == 0:
        count += 1
        continue
    count += 1
    print("{}:".format(file))
    weights_path = os.path.join(WEIGHTS_DIR, file)
    model.load_weights(weights_path, by_name=True)
    image_ids = np.random.choice(dataset_test.image_ids, 20)
    APs = []
    for j, image_id in enumerate(image_ids):
        # Load image and ground truth data
        image_path = dataset_test.image_info[image_id]['path']
        try:
            image, image_meta, gt_class_id, gt_bbox, gt_mask  =\
                modellib.load_image_gt(dataset_test, inference_config,
                                       image_id, use_mini_mask=False)
            # Run object detection
            results = model.detect([image], verbose=0)
            r = results[0]
            # Compute AP
            AP, precisions, recalls, overlaps =\
                utils.compute_ap(gt_bbox, gt_class_id, r["rois"], r["class_ids"], r["scores"])
            APs.append(AP)
            print('    {}: {}: {}: {}'.format(j + 1, image_id, image_path, AP))
        except:
            print("    Error processing image {}".format(image_path))
    mAP = np.mean(APs)
    mAPs.append(mAP)
    print("mAP: ", mAP)
plt.plot(mAPs)
plt.show()

mask_rcnn_seg_scenenn_0056.h5:
    1: 1838: /external_datasets/sceneNN/testing/213/image/image03618.png: 0.22857142984867096
    2: 1815: /external_datasets/sceneNN/testing/213/image/image01234.png: 0.08571428858808108
    3: 2552: /external_datasets/sceneNN/testing/069/image/image02867.png: 0.46785715594887733
    4: 2146: /external_datasets/sceneNN/testing/213/image/image05955.png: 0.2450980469584465
    5: 307: /external_datasets/sceneNN/testing/093/image/image04612.png: 0.20802005744051366
    6: 2080: /external_datasets/sceneNN/testing/213/image/image01193.png: 0.12023460052230141
    7: 2210: /external_datasets/sceneNN/testing/213/image/image03179.png: 0.1473484881730242
    8: 1425: /external_datasets/sceneNN/testing/082/image/image06863.png: 0.10526315867900848
    9: 388: /external_datasets/sceneNN/testing/093/image/image03043.png: 0.09994553728029132
    10: 944: /external_datasets/sceneNN/testing/082/image/image06399.png: 0.3076923191547394
    11: 869: /external_datasets/sc